# Name Screening
## Detect 50 Bad actors in our customer base using public data sources
**Task**: Find as many bad actors as possible using NLP techniques to match customer names with watchlist and other relevant information given.

In [2]:
import pandas as pd
import zipfile 
import warnings
warnings.filterwarnings("ignore")

In [3]:
# unzip data
with zipfile.ZipFile("data.zip","r") as zip_ref:
    zip_ref.extractall()

# load in relevant datasets
df_kyc = pd.read_csv("UofT_nodes.csv")
df_bad = pd.read_csv("targets.simple.csv", low_memory = False)

# retain relevant columns
df_kyc_cleaned = df_kyc[['NAME','BIRTH_DT']]
df_bad_cleaned = df_bad[['name','birth_date']]

df_kyc_cleaned.rename(columns={'NAME': 'name', 'BIRTH_DT': 'birth_date'}, inplace=True)

/Users/JennaFu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [3]:
# remove entried where name and birth_date are null for both datasets
df_kyc_cleaned = df_kyc_cleaned[~df_kyc_cleaned['name'].isnull()].reset_index(drop=True)
df_bad_cleaned = df_bad_cleaned[~df_bad_cleaned['name'].isnull()].reset_index(drop=True)

In [4]:
# standardize the format of the birth date columns
df_kyc_cleaned['birth_date_cleaned'] = pd.to_datetime(df_kyc_cleaned['birth_date'])
df_bad_cleaned['birth_date_cleaned'] = pd.to_datetime(df_bad_cleaned['birth_date'].str.\
                                                      split(';', n=1, expand = True)[0],
                                                      errors = 'coerce')

In [5]:
df_bad_cleaned.shape

(262426, 3)

In [6]:
df_kyc_cleaned.shape

(999340, 3)

## Method 1: String Grouper (Name + Birth Date Matched)
https://github.com/Bergvca/string_grouper

In [22]:
import pandas as pd
# update numpy if running into string_grouper import issue
# !pip install --upgrade numpy
from string_grouper import match_strings

In [23]:
matches = match_strings(df_kyc_cleaned['name'], df_bad_cleaned['name'], n_blocks = 'auto')
matches

/Users/JennaFu/opt/anaconda3/lib/python3.7/site-packages/string_grouper/string_grouper.py:492: UserWarning: An OverflowError occurred but is being handled.  The input data will be automatically split-up into smaller chunks which will then be processed one chunk at a time.  To prevent OverflowError, use the n_blocks parameter to split-up the data manually into small enough chunks.
  warnings.warn("An OverflowError occurred but is being "


,left_index,left_name,similarity,right_name,right_index
0,41139,David Robinson,0.847678,David ROBINSONOVA,4257
1,315806,David Robinson,0.847678,David ROBINSONOVA,4257
2,823753,David Robinson,0.847678,David ROBINSONOVA,4257
3,398002,James Mason,1.000000,James Mason,4482
4,759344,Daniel Martin,1.000000,Daniel Martin,4898
...,...,...,...,...,...
9529,793756,"Sullivan, George Ortiz",0.801988,George Ortiz,262195
9530,111485,Ana Gonzalez,0.808516,Deanna Gonzalez,262199
9531,274916,Jon Wyatt,0.802506,Brandon Wyatt,262206
9532,790486,Linda Callahan,0.850643,Amanda Callahan,262254


In [ ]:
matches_cosine = matches.merge(df_kyc_cleaned.reset_index()[['index','birth_date_cleaned']], 
                            left_on='kyc_index',right_on='index',how='left')\
                      .drop(['index'],axis=1)\
                      .rename(columns={'birth_date_cleaned': 'kyc_bt'})\
                      .merge(df_bad_cleaned.reset_index()[['index','birth_date_cleaned']], 
                              left_on='bad_index',right_on='index',how='left')\
                      .drop(['index'],axis=1)\
                      .rename(columns={'birth_date_cleaned': 'bad_bt'})

## Method 2: FuzzyWuzzy (Name + Birth Date Matched)

In [62]:
matches_fuzzy = pd.read_csv("matches_fuzzy.csv", index_col=0)

## Compile Results (Name + Birth Date Matched)

In [85]:
# Matched name and birth date from method 1
df_cosine_manual_matches = matches_cosine[matches_cosine['kyc_bt'] == matches_cosine['bad_bt']]

cosine_name_matches = df_cosine_manual_matches['kyc_name'].to_list()

In [86]:
# Matched name and birth date from method 2 THEN 1
cosine_fuzzy_matches = match_strings(matches_fuzzy['BadActor'], 
                                     matches_fuzzy['Found_Name'],
                                     n_blocks = 'auto')['right_Found_Name'].tolist()

In [87]:
# Matched name and birth date from method 2
# manual search through entries where Score is above 75
# matches_fuzzy[matches_fuzzy['Score'] >= 65]

fuzzy_manual_matches = [715 ,203, 916, 820, 48, 661, 1108, 739, 338, 1189, 
                        725, 606, 988, 401, 580, 1025, 102, 825, 721, 242,
                        740, 594, 741, 68, 909, 66, 1195, 317, 699, 291, 722,
                        757, 53, 489, 833]

df_fuzzy_manual_matches = matches_fuzzy.filter(items=fuzzy_manual_matches, axis=0)
fuzzy_name_matches = df_fuzzy_manual_matches['Found_Name'].to_list()

In [90]:
# Merge the two lists
df_kyc[df_kyc['NAME'].isin(fuzzy_name_matches + cosine_fuzzy_matches + cosine_name_matches)]\
.to_csv('badactor_foundin_kyc_bt_match.csv')